In [3]:
from keras.models import load_model
import joblib
import pandas as pd
import numpy as np 
import datetime as dt
import requests

DAYS = 50

In [4]:
model = load_model("model/lstm_bnf_model.h5")
scaler = joblib.load('model/scaler.gz')

In [5]:
def get_1h_url(sym='23', res='1D', days= DAYS):
    t_to = str(int(dt.datetime.now().timestamp()))
    t_from = str(int((dt.datetime.now()-dt.timedelta(DAYS)).timestamp()))
    url = "https://priceapi.moneycontrol.com/techCharts/history?symbol="+\
            sym+"&resolution="+res+"&from="+t_from+"&to="+t_to
    return url

def get_bnf_data():
    r = requests.get(get_1h_url())
    data = r.json()
    df = pd.DataFrame(data)
    def convert_to_date(tsmp):
        return dt.datetime.fromtimestamp(tsmp)

    df['t'] = df['t'].apply(convert_to_date)
    df.sort_values(['t'], inplace=True, ascending=False)
    df.drop(['s','t','o','l','h','v'], axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

bnf = get_bnf_data()
bnf = bnf.iloc[::-1]

In [6]:
'''Function to create a dataset to feed into an LSTM'''
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [7]:
# test data
time_steps = 20

test_data = bnf['c'].values
test_data = test_data.reshape(-1,1)
test_data = scaler.transform(test_data)

# Create the data to test our model on:
X_test, y_test = create_dataset(test_data, time_steps)

# store the original vals for plotting the predictions 
y_test = y_test.reshape(-1,1)
org_y = scaler.inverse_transform(y_test)

# reshape it [samples, time steps, features]
X_test = np.reshape(X_test, (X_test.shape[0], time_steps, 1))

In [8]:
# Predict the prices with the model
predicted_y = model.predict(X_test)

predicted_y = scaler.inverse_transform(predicted_y)
test_data = scaler.inverse_transform(test_data)

In [9]:
res = pd.DataFrame(org_y, columns=['Input'])
res['Prediction'] = pd.Series(predicted_y.ravel())

res['prev_pred'] = res['Prediction'].shift(1)
res['prev_close'] = res['Input'].shift(1)

def get_direction(x, col1, col2):
    if x[col1] - x[col2] > 0:
        return x[col1] - x[col2], "UP"
    elif x[col1] - x[col2] < 0:
        return x[col1] - x[col2], "DOWN"
    else:
        return 0, "NEUTRAL"

res[['pred_Diff','pred_Direction']] = res.apply(lambda x: get_direction(x,'Prediction','prev_pred'), axis=1, result_type='expand')
res[['Actual_Diff','Actual_Direction']] = res.apply(lambda x: get_direction(x,'Input','prev_close'), axis=1, result_type='expand')
res[['Input','prev_close', 'Actual_Diff', 'Actual_Direction', 'Prediction', 'prev_pred', 'pred_Diff', 'pred_Direction']]

,Input,prev_close,Actual_Diff,Actual_Direction,Prediction,prev_pred,pred_Diff,pred_Direction
0,37741.00,NaN,0.00,NEUTRAL,37466.152344,NaN,0.000000,NEUTRAL
1,37521.55,37741.00,-219.45,DOWN,37596.976562,37466.152344,130.824219,UP
2,37753.20,37521.55,231.65,UP,37512.109375,37596.976562,-84.867188,DOWN
3,37775.25,37753.20,22.05,UP,37605.191406,37512.109375,93.082031,UP
4,38293.80,37775.25,518.55,UP,37651.640625,37605.191406,46.449219,UP
5,38521.50,38293.80,227.70,UP,37951.750000,37651.640625,300.109375,UP
6,38635.75,38521.50,114.25,UP,38177.671875,37951.750000,225.921875,UP
7,39340.90,38635.75,705.15,UP,38318.152344,38177.671875,140.480469,UP
8,39684.80,39340.90,343.90,UP,38756.378906,38318.152344,438.226562,UP
9,39540.50,39684.80,-144.30,DOWN,39099.171875,38756.378906,342.792969,UP
